In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import cv2
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.utils import plot_model, img_to_array
from tensorflow.keras.models import load_model

from IPython import display
from PIL import Image


In [ ]:
high_res_train_images_path = "/kaggle/input/image-super-resolution/dataset/train/high_res"
low_res_train_images_path = "/kaggle/input/image-super-resolution/dataset/train/low_res"

In [ ]:
high_res_test_images_path = "/kaggle/input/image-super-resolution/dataset/val/high_res"
low_res_test_images_path = "/kaggle/input/image-super-resolution/dataset/val/low_res"

In [ ]:
SIZE = 256

In [ ]:
#Function for loading images
def load_images(path, size=224, count=None):
    if count == None:
        files = os.listdir(path)
    else:
        files = os.listdir(path)[:count]
    images = []

    for file in tqdm.tqdm(files):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (size, size))
        img = img.astype("float32") / 255.
        img = img_to_array(img)
        images.append(img)

    images = np.array(images)
    return images

In [ ]:
train_low_images = load_images(low_res_train_images_path, size=SIZE)
train_high_images = load_images(high_res_train_images_path, size=SIZE)

In [ ]:
#Function for plotting random images from training dataset
def plot_random_images(low_res_images, high_res_images, num_images=5, figsize=(12, 8)):
    fig, axes = plt.subplots(nrows=num_images, ncols=2, figsize=figsize)
    
    random_indices = random.sample(range(len(low_res_images)), num_images)
    
    for i, idx in enumerate(random_indices):
        axes[i, 0].imshow(low_res_images[idx])
        axes[i, 0].set_title(f"Low-Res Image {idx}")
        axes[i, 0].axis("off")
        
        axes[i, 1].imshow(high_res_images[idx])
        axes[i, 1].set_title(f"High-Res Image {idx}")
        axes[i, 1].axis("off")
    
    plt.tight_layout()
    plt.show()


In [ ]:
plot_random_images(train_low_images, train_high_images, num_images=5, figsize=(12, 10))

In [ ]:
#CNN MODEL
# def down_block(x, filters, kernel_size, apply_batch_normalization=True):
#     x = layers.Conv2D(filters, kernel_size, padding="same", strides=2)(x)
#     if apply_batch_normalization:
#         x = layers.BatchNormalization()(x)

#     x = layers.LeakyReLU()(x)
#     return x
    
# def up_block(x, skip, filters, kernel_size, dropout=False):
#     x = layers.Conv2DTranspose(filters, kernel_size, padding="same", strides=2)(x)
#     if dropout:
#         x = layers.Dropout(0.1)(x)

#     x = layers.LeakyReLU()(x)
#     x = layers.concatenate([x, skip])
#     return x

In [ ]:
# def build_model(size):
#     inputs = layers.Input(shape=[size, size, 3])

#     # Downsampling
#     d1 = down_block(inputs, 128, (3, 3), apply_batch_normalization=False)
#     d2 = down_block(d1, 128, (3, 3), apply_batch_normalization=False)
#     d3 = down_block(d2, 256, (3, 3), apply_batch_normalization=True)
#     d4 = down_block(d3, 512, (3, 3), apply_batch_normalization=True)
#     d5 = down_block(d4, 512, (3, 3), apply_batch_normalization=True)

#     # Upsampling
#     u1 = up_block(d5, d4, 512, (3, 3), dropout=False)
#     u2 = up_block(u1, d3, 256, (3, 3), dropout=False)
#     u3 = up_block(u2, d2, 128, (3, 3), dropout=False)
#     u4 = up_block(u3, d1, 128, (3, 3), dropout=False)

#     # Final upsampling
#     u5 = layers.Conv2DTranspose(3, (3, 3), padding='same', strides=2)(u4)
#     u5 = layers.LeakyReLU()(u5)
#     u5 = layers.concatenate([u5, inputs])

#     # Output layer
#     outputs = layers.Conv2D(3, (2, 2), padding='same', strides=1)(u5)
#     return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# model = build_model(size=SIZE)

In [ ]:
# model.summary()

In [ ]:
# model.compile(
#     optimizer = optimizers.Adam(learning_rate=0.001),
#     loss = "mean_absolute_error",
#     metrics = ["accuracy"]
# )

In [ ]:
# history = model.fit(
#     train_low_images,
#     train_high_images,
#     epochs = 100,
#     batch_size = 16,
#     validation_data=(test_low_images, test_high_images),
#     verbose = 1
# )

In [ ]:
# model.save('/kaggle/working/img_resolution.h5')

In [ ]:
trained_model=load_model('/kaggle/input/imgresolution/tensorflow2/default/1/img_resolution.h5')

In [ ]:
trained_model.summary()

In [ ]:
test_low_images = load_images(low_res_test_images_path, size=SIZE)
test_high_images = load_images(high_res_test_images_path, size=SIZE)

In [ ]:
test_loss, test_accuracy =trained_model.evaluate(test_low_images,test_high_images)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
def show_images(test_low, test_high, count=10, size=256):
    for _ in range(count):
        random_idx = np.random.randint(len(test_low))
        predicted = trained_model.predict(test_low[random_idx].reshape(1, size, size, 3), verbose=0)
        predicted = np.clip(predicted, 0.0, 1.0).reshape(size, size, 3)
        
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
        
        axes[0].imshow(test_low[random_idx])
        axes[0].set_title("Low-Resolution Image")
        axes[0].axis("off")
        
        axes[1].imshow(test_high[random_idx])
        axes[1].set_title("Ground Truth")
        axes[1].axis("off")

        axes[2].imshow(predicted)
        axes[2].set_title("Enhanced Image")
        axes[2].axis("off")
        
        plt.show()

In [ ]:
show_images(test_low_images, test_high_images, count=10, size=SIZE)

In [ ]:
input ="/kaggle/input/final-testing/test5_img.png"

In [ ]:
import cv2 as cv
img = cv.imread(input, cv.IMREAD_COLOR) 
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.resize(img, (256, 256))
img = img.astype('float32') / 255.0

img_tensortest = tf.convert_to_tensor(img, dtype=tf.float32)

In [ ]:
if isinstance(input, str):  # If it's a file path
    input_img = cv.cvtColor(cv.imread(input), cv.COLOR_BGR2RGB)
    input_img = input_img.astype('float32') / 255.0  # Normalize to [0, 1]
elif isinstance(input, np.ndarray):  # If it's already an array
    input_img = (input * 255).astype('uint8')
else:  # Handle tensors
    input_img = (input.numpy() * 255).astype('uint8')

In [ ]:
enhanced = trained_model.predict(np.expand_dims(img_tensortest, axis=0))[0]

In [ ]:
def process_model_output(model_output):
    if isinstance(model_output, np.ndarray):  # If it's already an array
        return (model_output * 255).astype('uint8')
    elif hasattr(model_output, 'numpy'):  # If it's a tensor
        return (model_output.numpy() * 255).astype('uint8')
    else:
        raise ValueError("Unexpected type for model output.")

enhanced_img = process_model_output(enhanced)

In [ ]:
def show_images(input,output):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title("Input")
    plt.imshow(input)
    plt.axis('off')  

    plt.subplot(1, 3, 2)
    plt.title("Output")
    plt.imshow(output)
    plt.axis('off')

    plt.show()

In [ ]:
show_images(input_img,enhanced)